# Exercise 01

**Objetivo:** Resolver o seguinte subproblema de região de confiança.

$$
\max x^TCx
$$

Sujeito a:

$$
x^Tx \leq 1
$$

## Imports

In [1]:
import numpy as np
from scipy.io import savemat
import gurobipy as gp
from gurobipy import GRB

## Set Up

### Creating symmetric matrix

In [2]:
# Create a new symmetric positive semidefinite matrix
n = 10
C = np.random.rand(n, n)
C = np.matmul(C, C.transpose())

In [3]:
# Name of file .mat
filename = 'matrixC.mat'

# Save numpy array as a file .mat
savemat(filename, {'matrixC': C})

### Expected solution

In [4]:
# Calculates eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(C)

# The expected value of the objective function is the biggest eigenvalue
phi = max(eigenvalues)

# And the expected value of x is the corresponding eigenvector
v = eigenvectors[np.where(eigenvalues == phi)][0]

print("Biggest eigenvalue: ", phi)
print("Corresponding eigenvector: ", v)

Biggest eigenvalue:  20.105238123401083
Corresponding eigenvector:  [ 0.25111285  0.05560033 -0.40878978  0.21027457 -0.50020946 -0.14398314
 -0.2445416   0.30696896 -0.4307971  -0.3346179 ]


### Setting Model

In [5]:
# Create a new model
model = gp.Model("quadratic")

# Create variables
x = model.addVars(n, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")

# Define objective function
objective = gp.quicksum(C[i, j] * x[i] * x[j] for i in range(n) for j in range(n))

# Set objective
model.setObjective(objective, GRB.MAXIMIZE)

# Add constraint: x^Tx <= 1
model.addConstr(gp.quicksum(x[i] * x[i] for i in range(n)) <= 1, name="constraint")

Restricted license - for non-production use only - expires 2025-11-24


<gurobi.QConstr Not Yet Added>

### Optimizing model

In [6]:
# Optimize model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 10 columns and 0 nonzeros
Model fingerprint: 0xd07473ee
Model has 55 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+00, 1e+00]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 122 rows, 66 columns, 386 nonzeros
Presolved model has 55 bilinear constraint(s)
Variable types: 66 continuous, 0 integer (0 binary)

Root relaxation: objective 1.730605e+02, 39 i

### Checking optimizing results

In [7]:
# Printing solution
for var in model.getVars():
    print(f"{var.VarName} {var.X:g}")

# Printing objective function value
print(f"Obj: {model.ObjVal:g}")

x[0] 0.251113
x[1] 0.269954
x[2] 0.370593
x[3] 0.322599
x[4] 0.354135
x[5] 0.209199
x[6] 0.369303
x[7] 0.304569
x[8] 0.318475
x[9] 0.350584
Obj: 20.1052


### Comparing results found with expected results

In [8]:
epsilon = 10 ** -2

def is_multiple(x, y):
    c = 0
    if y[0] != 0:
        c = x[0] / y[0]
        y = c * y
    else:
        c = y[0] / x[0]
        x = c * x
    error = 0
    for i in range(len(x)):
        error += np.square(x[i]-y[i])
    print("Error:", error)
    if error < epsilon:
        return True
    return False

if (model.ObjVal - phi)**2 < epsilon:
    print("Objetive function value is the expected value.")
else:
    print("Objective function value is not the expected value.")

x = np.array([x.x for x in model.getVars()])

if is_multiple(x, v):
    print("Solution vector is the expected value.")
else:
    print("Solution vector is not the expected value.")

Objetive function value is the expected value.
Error: 2.9283644144345793
Solution vector is not the expected value.
